# Dataset
 - Fazer download do dataset
 - Plotar um conjunto de imagens para ver o que elas se parecem e quais os labels que ela recebe.

Kaggle Dataset = Planet Understanding the Amazon from Space

[Planet: Understanting the Amazon from Space](https://www.kaggle.com/c/planet-understanding-the-amazon-from-space)

 - Tensorflow: Usar a função de tk.keras.util ou tf.io ou tf.data para fazer o download do arquivo e unzip dele.
 - Sabendo que cada imagem tem mais de 1 lable, como iremos montar nosso dataset? Como o tf.data interpreta múltiplos labels?
 - Como o Keras no modelo irá tratar isso em termos do Loss e Optimizer?

Labels
 - Carregar o CSV e fazer um print para entender como está estruturado os labels.
 - Criar um dataset desse conjunto de imagens e labels.

tf.Data
 - Criar o dataset com as imagens e labels.
 - Para cada imagem aplicar as seguintes transformações:
  - Flip Vertical
  - Max Lightining = 0.1
  - Max Zoom = 1.05
  - Max Warp = 0.
 - Split de 20% do dataset para validação (deixar umas imagens de fora para test)
 - Batch Size de 128 imagens
 - Repeat do dataset e determinar o número de steps
 - Normalizar para entre 0 e 1.
 
Nesse dataset, como faço para determinar a quantidade de imagens, quantos labels existem, quais as classes, etc.?

 - Como fazemos para criar datasets para Object Detection? Image Segmentation? Multi label? Text? Corpus?

### Future Model

Criar um modelo para reconhecer o que é um prédio, uma casa, uma casa em construção, um barrac, etc.
Criar um dataset com o Planet para detectar isso.

# Model
 - Usar o ResNet50 como base

Métricas Customizadas
 - Accuracy
 - F-Score
  - Relembrar o que é Precission, Accuracy, Recall e F(n)-Score
 - Threashold de 0.2 para o FScore

Método
 - Model.fit 
 - Gradient Tapes

?? Num modelo de classificação, como coloco um threashold na classificação.? Isso seria no Loss? Ou em alguma métrica específica?

Learning Rate
 - Achar o LR ideal baseado no decay do plot da função do LR x epochs.

Callbacks
 - Criar os seguintes callbacks (quando apropriado):
  - ModelSaveweights
  - LearningRateScheduler
  - EarlyStop

Save
 - Salvar o modelo em um arquivo .h5, checkpoint e SavedModel.
 - Carregar todos eles e lembrar as diferenças entre eles.

### Performance

 - Com ResNet50 devemos ter pelo menos 92% de F2-Score.
  - Como melhorar isso?
 - Podemos treinar o resto da rede para customizar ainda mais o resultado.
 - Mudar o batch size e fazer o RESHAPE para 256 x 256 da imagem.

Começar o treinamento com imagens 128 x 128 e usar os mesmos weights já treinados para imagens com 256 x 256.




# Imports

In [2]:
import tensorflow as tf

In [3]:
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# Read filenames and labels
ds_train = pd.read_csv('/root/.fastai/data/planet_sample/labels.csv')

# Print first 5 lines
ds_train.head()

# Prepare Dataset for Tensorflow

In [ ]:
# Create a list of unique labels (set)
labels = set()
for i in ds_train['tags']:
    labels.update(i.split(sep=' '))

labels = list(labels)
labels.sort()

In [ ]:
def create_tag_mapping(ds_train):
    labels = set()
    for i in ds_train['tags']:
        labels.update(i.split(sep=' '))

    labels = list(labels)
    labels.sort()

    label_map = {labels[i]:i for i in range(len(labels))}
    inv_label_map = {i:labels[i] for i in range(len(labels))}

    return label_map, inv_label_map

In [ ]:
def create_file_mapping(ds_train):
    mapping = dict()
    for i in range(len(ds_train)):
        name, tags = ds_train['image_name'][i], ds_train['tags'][i]
        name = '/root/.fastai/data/planet_sample/train/' + name + '.jpg'
        mapping[name] = tags.split(' ')
    return mapping

In [ ]:
def one_hot_encode(tag_mapping, file_mapping):
    dataset = dict()
    for filename, tags in file_mapping.items():
        encoding = np.zeros(len(tag_mapping), dtype='uint8')
        for tag in tags:
            encoding[tag_mapping[tag]] = 1
        dataset.update({filename:list(encoding)})
    
    return dataset

In [ ]:
tag_mapping, inv_label_map = create_tag_mapping(ds_train)
file_mapping = create_file_mapping(ds_train)

In [ ]:
dataset = one_hot_encode(tag_mapping, file_mapping)
dataset = [[k,v] for k,v in dataset.items()]

In [ ]:
features = [i[0] for i in dataset]
labels = [i[1] for i in dataset]

In [ ]:
# Quick function to convert image to jpg
def convert_to_jpg(filename):
    im = Image.open(filename)
    rgb_im = im.convert('RGB')
    rgb_im.save(filename)

In [ ]:
# Loop over list with filenames and convert to jpeg
for filename in features:
    convert_to_jpg(filename)

# Create Tf.data.dataset

In [ ]:
tf_dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
def preprocess_data(feature, label):
    # Load raw data to uint8
    img = tf.io.read_file(feature)
    # convert jpeg to uint tensor
    img = tf.image.decode_jpeg(img)
    # convert to float in [0,1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to desired size
    img = tf.image.resize(img, [224,224])
    
    return img, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
labeled_ds = tf_dataset.map(preprocess_data, num_parallel_calls=AUTOTUNE)

In [ ]:
def prepare_training(ds, cache=True, shuffle_buffer_size=1000):
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat().batch(32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

In [ ]:
train_ds = prepare_training(labeled_ds)

In [ ]:
def show_batch(image, label):
    plt.figure(figsize=(10,10))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image[n])
        plt.axis('off')

# Quick Visualization of Labels

In [ ]:
image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy())